In [193]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

#create database
kill_match_db = create_engine('sqlite:///kill_match.db')
df1 = pd.read_sql_query('select killed_by, killer_position_x, killer_position_y, victim_position_x, victim_position_y from kmatch where killed_by not like "%death.%" and killed_by not like "%Down%" and killed_by not like "%Blue%" and killed_by not like "%Fall%" and map = "MIRAMAR" and killed_by like "SKS" limit 10000', kill_match_db)




In [194]:
df1['kill_dist'] = ((df1['killer_position_x'] - df1['victim_position_x'])**2 + 
                    (df1['killer_position_y'] - df1['victim_position_y'])**2)**(1/2)
df1

,killed_by,killer_position_x,killer_position_y,victim_position_x,victim_position_y,kill_dist
0,SKS,421914.3,276550.0,408728.8,293670.3,21609.305457
1,SKS,436131.1,280898.6,435692.5,280670.9,494.183417
2,SKS,427291.3,528331.9,425796.0,525071.3,3587.120635
3,SKS,139546.3,602450.0,139464.2,602319.6,154.092732
4,SKS,329325.9,273781.9,324836.3,270891.0,5339.832485
5,SKS,319972.2,273620.9,315237.3,275874.0,5243.637823
6,SKS,347721.2,266362.4,350504.1,270965.6,5379.031758
7,SKS,322614.3,525392.2,313699.4,516623.9,12504.340322
8,SKS,374958.9,293157.2,375020.0,293086.8,93.216790
9,SKS,443139.1,360772.6,442095.2,361087.8,1090.448646


In [195]:
def Kmean(k):
    cluster_centers = []
    convergence = 0
    temp = 0
    group0 = []
    group1 = []
    group2 = []
    group3 = []
    
    #Random choose k centers
    for i in range (k):
        index = np.random.choice(len(df1['kill_dist']))
        cluster_centers.append(round(df1.loc[index,'kill_dist']))
    
    #Main loop
    while convergence !=k:
        convergence = 0
        groups = []
        
        #find the closest center for each point
        for i in range(len(df1['kill_dist'])):
            min_dist = abs(df1.loc[i,'kill_dist'] - cluster_centers[0])
            min_ind = 0
            for j in range(1,k):
                temp_dist = abs(df1.loc[i,'kill_dist'] - cluster_centers[j])
                if temp_dist < min_dist:
                    min_dist = temp_dist
                    min_ind = j
            groups.append(min_ind)
        
        #calculate the new center
        for i in range (k):
            dsum = 0
            count = 0
            for j in range(len(df1['kill_dist'])):
                if groups[j] == i:
                    dsum += df1.loc[j,'kill_dist']
                    count += 1
            new_center = dsum/count
            if (new_center == cluster_centers[i]):
                convergence += 1
            else:
                cluster_centers[i] = new_center
    
    #Storing the final result into list
    for i in range(len(df1['kill_dist'])):
        if groups[i] == 0:
            group0.append(int(round(df1.loc[i,'kill_dist']))/100)
        elif groups[i] == 1:
            group1.append(int(round(df1.loc[i,'kill_dist']))/100)
        elif groups[i] == 2:
            group2.append(int(round(df1.loc[i,'kill_dist']))/100)
        else:
            group3.append(int(round(df1.loc[i,'kill_dist']))/100)
    
    #Rounding the centers
    for i in range (len(cluster_centers)):
        cluster_centers[i] = int(round(cluster_centers[i]))/100
        
    max0 = max(group0)
    min0 = min(group0)
    max1 = max(group1)
    min1 = min(group1)
    max2 = max(group2)
    min2 = min(group2)
    max3 = max(group3)
    min3 = min(group3)
    count0 = len(group0)
    count1 = len(group1)
    count2 = len(group2)
    count3 = len(group3)
    
    print ("Cluster 0:","\n", "center: ",cluster_centers[0],"\n","count: ", count0,"\n","range: (",min0,",",max0,")")
    print ("\n")
    print ("Cluster 1:","\n", "center: ",cluster_centers[1],"\n","count: ", count1,"\n","range: (",min1,",",max1,")")
    print ("\n")
    print ("Cluster 2:","\n", "center: ",cluster_centers[2],"\n","count: ", count2,"\n","range: (",min2,",",max2,")")
    print ("\n")
    print ("Cluster 3:","\n", "center: ",cluster_centers[3],"\n","count: ", count3,"\n","range: (",min3,",",max3,")")

            
                


In [196]:
Kmean(4)

Cluster 0: 
 center:  248.09 
 count:  1452 
 range: ( 185.49 , 381.7 )


Cluster 1: 
 center:  27.66 
 count:  4875 
 range: ( 0.0 , 75.21 )


Cluster 2: 
 center:  122.79 
 count:  3526 
 range: ( 75.25 , 185.35 )


Cluster 3: 
 center:  517.57 
 count:  147 
 range: ( 382.9 , 971.99 )
